In [1]:
import sqlalchemy as db
from faker import Faker
from faker_food import FoodProvider
import random as random


In [2]:
faker= Faker()
faker.add_provider(FoodProvider)

In [3]:
faker.name()

'Ronald Johnson'

In [4]:
list_pays=[]
for i in range(100):
   list_pays.append(faker.country())
print(list_pays)

['Syrian Arab Republic', 'Samoa', 'Ecuador', 'Barbados', 'Bosnia and Herzegovina', 'Turkmenistan', 'Liberia', 'South Georgia and the South Sandwich Islands', 'Cook Islands', 'Dominican Republic', 'Anguilla', 'Serbia', 'Georgia', 'Burundi', 'France', 'Burkina Faso', 'Philippines', 'Tuvalu', 'Turkey', 'Eritrea', 'Dominican Republic', 'Greenland', 'Georgia', 'Ethiopia', 'Ireland', 'Ireland', 'Tajikistan', 'Saint Vincent and the Grenadines', 'Brazil', 'Thailand', 'Canada', 'Tuvalu', 'Cameroon', 'Thailand', 'Saint Martin', 'Vietnam', 'Paraguay', 'Palau', 'Azerbaijan', 'Samoa', 'Suriname', 'Turkmenistan', 'Vietnam', 'Tonga', 'China', 'Montenegro', 'Burkina Faso', 'Poland', 'Turkmenistan', 'Denmark', 'Hungary', 'Isle of Man', 'Fiji', 'Ireland', 'Singapore', 'Fiji', 'Sweden', 'Mauritania', 'Panama', 'Switzerland', 'Benin', 'Somalia', 'Maldives', 'Peru', 'Lithuania', 'Argentina', 'Bulgaria', 'Greenland', 'Palestinian Territory', 'Saint Martin', 'Kyrgyz Republic', 'Somalia', 'Jordan', 'Korea', '

In [5]:
faker.address()

'800 Davis Key Apt. 215\nPort Michaelburgh, AR 37047'

In [6]:
faker.text()

'Dinner meeting party media add and. Price recognize president forward man. Buy various method mission look play edge start.'

In [7]:
#fake.dish()
#fake.fruit()
#fake.spice()
#fake.vegetable()
#fake.ingredient()

In [8]:
# sqlite://<nohostname>/<path>
# where <path> is relative:
engine = db.create_engine("sqlite:///restaurant.db")

In [9]:
connection = engine.connect()
metadata = db.MetaData()


In [10]:


ingredient_table = db.Table('Ingredient', metadata,
              db.Column('nom_ingredient', db.String(), primary_key=True),
              db.Column('prix_achat', db.Float(), default=0.0),
              )



In [11]:


pays_table = db.Table('Pays', metadata,
              db.Column('pays', db.String(), primary_key=True),
              )



In [12]:


restaurant_table = db.Table('Restaurant', metadata,
              db.Column('code_postal', db.String(), primary_key=True),
              db.Column('parking', db.Integer(),nullable=False),
              db.Column('espace_enfant', db.Integer(),default=0),
              db.Column('pmr', db.Integer(),default=0),
              db.Column('borne', db.Integer(),default=0),
              db.Column('capacite', db.Integer(),default=0),
              db.Column('pays', db.String(), db.ForeignKey("Pays.pays"),nullable=False),
              )



In [13]:


stock_table = db.Table('Stock', metadata,
              db.Column('id_stock', db.Integer(),primary_key=True),
              db.Column('nom_ingredient', db.String(),db.ForeignKey("Ingredient.nom_ingredient"),nullable=False),
              db.Column('code_postal', db.String(),db.ForeignKey("Restaurant.code_postal"),nullable=False),
              db.Column('quantite', db.Integer(),default=0),
              )



In [14]:


item_table = db.Table('Item', metadata,
              db.Column('nom_item', db.String(),primary_key=True),
              db.Column('type', db.String(),nullable=False),
              db.Column('prix', db.Float(),default=0.00),
              )



In [15]:

carte_item_table = db.Table('CarteItem', metadata,
              db.Column('id_carte_item', db.Integer(),primary_key=True),
              db.Column('nom_item', db.String(),db.ForeignKey("Item.nom_item"),nullable=False),
              db.Column('pays', db.String(),db.ForeignKey("Pays.pays"),nullable=False),
              )

In [16]:
employes_table= db.Table ('Employes', metadata,
            db.Column('id_employes',db.Integer(),primary_key=True),
            db.Column('nom',db.String(),nullable=False),
            db.Column('role',db.String(),nullable=False),
            db.Column('id_superieur',db.Integer(),db.ForeignKey("Employes.id_employes")),
            db.Column('anciennete',db.Integer(),default=0),
            db.Column('compte_bancaire',db.Integer()),
            db.Column('note',db.Integer()),
            db.Column('adresse',db.String()),
            db.Column('code_postal',db.String(),db.ForeignKey("Restaurant.code_postal"),nullable=False),
            )

In [17]:
bulletin_paye_table = db.Table('BulletinPaye', metadata,
              db.Column('id_employes', db.Integer(),db.ForeignKey("Employes.id_employes"),primary_key=True),
              db.Column('date', db.String(),primary_key=True,nullable=False),
              db.Column('salaire', db.Float(),nullable=False),
               db.Column('poste', db.String(),nullable=False),
              )

In [18]:
recette_table = db.Table('Recette', metadata,
        db.Column('id_recette', db.Integer(),primary_key=True),
        db.Column('nom_item', db.String(),db.ForeignKey("Item.nom_item"),nullable=False),
        db.Column('nom_ingredient', db.String(),db.ForeignKey("Ingredient.nom_ingredient"),nullable=False),
        db.Column('quantite_ingredient', db.Integer(),default=0)
        )

In [19]:
menu_table = db.Table('Menu', metadata,
        db.Column('nom_menu', db.String(),primary_key=True),
        db.Column('plat', db.String(),db.ForeignKey("Item.nom_item"),nullable=False),
        db.Column('dessert', db.String(),db.ForeignKey("Item.nom_item"),nullable=False),
        db.Column('boisson', db.String(),db.ForeignKey("Item.nom_item"),nullable=False),
        db.Column('prix', db.Integer(),default=0.00)
        )

In [20]:
carte_menu_table = db.Table('CarteMenu', metadata,
              db.Column('id_carte_menu', db.Integer(),primary_key=True),
              db.Column('nom_menu', db.String(),db.ForeignKey("Item.nom_item"),nullable=False),
              db.Column('pays', db.String(),db.ForeignKey("Pays.pays"),nullable=False),
              )

In [21]:
addition_table = db.Table('Addition', metadata,
              db.Column('id_ticket', db.Integer(),primary_key=True),
              db.Column('heure', db.Integer(),nullable=False),
              db.Column('code_postal', db.String(),db.ForeignKey("Restaurant.code_postal"),nullable=False),
              db.Column('id_employes', db.String(),db.ForeignKey("Employes.id_employes"),nullable=False),
              db.Column('somme', db.REAL(),default=0),
              db.Column('moyen_paiment', db.String(),nullable=False),
              )

In [22]:
panier_item_table = db.Table('PanierItem', metadata,
              db.Column('id_ticket', db.Integer(),db.ForeignKey("Addition.id_ticket"),primary_key=True),
              db.Column('nom_item', db.Integer(),db.ForeignKey("Item.nom_item"),primary_key=True),
              db.Column('quantite_item', db.Integer(),default=0),
              )

In [23]:
panier_menu_table = db.Table('PanierMenu', metadata,
              db.Column('id_panier_menu', db.Integer(), primary_key=True),
              db.Column('id_ticket', db.Integer(),db.ForeignKey("Addition.id_ticket"),nullable=False),
              db.Column('nom_menu', db.Integer(),db.ForeignKey("Menu.nom_menu"),nullable=False),
              db.Column('quantite_menu', db.Integer(),default=0),
              )

In [24]:
metadata.create_all(engine)

In [25]:
list_pays=[]
with engine.begin() as connection:
    for _ in range(10):
      insert_stmt = pays_table.insert().values(
        pays= faker.country(),
       )
      connection.execute(insert_stmt)

In [26]:
with engine.begin() as connection:
  pays_pk = connection.execute(db.select([pays_table.c.pays])).fetchall()

with engine.begin() as connection:
     for i in range(50):
       aleatoire=random.randint(0,len(pays_pk)-1)
       insert_stmt = restaurant_table.insert().values(
         code_postal= random.randint(10000,99999),
         capacite= random.randint(30,100),
         parking= random.randint(0,1),
         espace_enfant= random.randint(0,1),
         pmr=random.randint(0,1),
         borne=random.randint(0,1),
         pays=pays_pk[aleatoire][0],
        )
       connection.execute(insert_stmt)


In [27]:
print(pays_pk)
print(random.choice(pays_pk))
print(pays_pk[1])
print(pays_pk[1][0])
print(pays_pk[aleatoire][0])
print()

[('Tuvalu',), ('Jersey',), ('Saint Pierre and Miquelon',), ('El Salvador',), ('Montserrat',), ('Uruguay',), ('Vanuatu',), ('Taiwan',), ('Timor-Leste',), ('Belize',)]
('Timor-Leste',)
('Jersey',)
Jersey
Vanuatu



In [28]:
List_role=['manager','director','caissier','cuisinier']
List_employes=[]
for i in range(26):
    if i==0:
        List_employes.append(List_role[1])
    if i>0 and i<=3:
        List_employes.append(List_role[0])
    if i>4 and i<=17:
        List_employes.append(List_role[3])
    if i>16 :
        List_employes.append(List_role[2])
     
print(List_employes)

['director', 'manager', 'manager', 'manager', 'cuisinier', 'cuisinier', 'cuisinier', 'cuisinier', 'cuisinier', 'cuisinier', 'cuisinier', 'cuisinier', 'cuisinier', 'cuisinier', 'cuisinier', 'cuisinier', 'cuisinier', 'caissier', 'caissier', 'caissier', 'caissier', 'caissier', 'caissier', 'caissier', 'caissier', 'caissier']


In [29]:
# with engine.begin() as connection:
#   employes_pk = connection.execute(db.select([employes_table.c.pays])).fetchall()
# e=0
# with engine.begin() as connection:
#     for i in range(26):
         
#       insert_stmt = employes_table.insert().values(
#         id_employes= i,
#         nom= faker.name(),
#         role=List_employes[i],
#         id_superieur = i+1,
#         anciennete= randint(0,10),
#         compte_bancaire=randint(10000000,99999999),
#         note=randint(0,10),
#         adresse=faker.address(),
#         code_postal=randint(10000,99999)
       
#        )
#       connection.execute(insert_stmt)

AttributeError: pays

In [30]:
# with engine.begin() as connection:
#     for i in range(26):
#       insert_stmt = bulletin_paye_table.insert().values(
#         id_employes= i,
#         date= faker.date(),
#         salaire=randint(1200,2500),
#         poste =List_employes[i],
       
#        )
#       connection.execute(insert_stmt)

NameError: name 'randint' is not defined

In [31]:
# with engine.begin() as connection:
#     for i in range(26):
#       insert_stmt = bulletin_paye_table.insert().values(
#         id_employes= i,
#         date= faker.date(),
#         salaire=randint(1200,2500),
#         poste =List_employes[i],
       
#        )
#       connection.execute(insert_stmt)

NameError: name 'randint' is not defined

In [ ]:
type_liste = ["boisson", "dessert", "plat"]
with engine.begin() as connection:
    for _ in range(30):
      insert_stmt = item_table.insert().values(
        nom_item = faker.unique.dish(),
        type = random.choice(type_liste),
        prix = round(random.uniform(2, 10),2),

      )
      connection.execute(insert_stmt)

In [ ]:
with engine.begin() as connection:
    for _ in range(40):
      insert_stmt = ingredient_table.insert().values(
          nom_ingredient = faker.unique.ingredient(),
          prix_achat = round(random.uniform(0.2,8),2)

      )
      connection.execute(insert_stmt)

In [ ]:
with engine.begin() as connection:
    item_pk = connection.execute(db.select([item_table.c.nom_item])).fetchall()
with engine.begin() as connection:
    ingredient_pk = connection.execute(db.select([ingredient_table.c.nom_ingredient])).fetchall()


with engine.begin() as connection:
    for i in range(len(item_pk)):
        ingredient_selectionne = []
        for nb_ingredient in range(random.randrange(1,6)):
            ingre = random.choice([ing for ing in ingredient_pk if ing not in ingredient_selectionne])[0]
            insert_stmt = recette_table.insert().values(
                
                nom_item = item_pk[i][0],
                nom_ingredient = ingre,
                
                quantite_ingredient = random.randrange(1,5),
            )
            ingredient_selectionne.append(ingre)
            
            connection.execute(insert_stmt)

In [ ]:
with engine.begin() as connection:
    item_plat_pk = connection.execute(db.select([item_table.c.nom_item]).where(item_table.c.type=="plat")).fetchall()
with engine.begin() as connection:
    item_dessert_pk = connection.execute(db.select([item_table.c.nom_item]).where(item_table.c.type =="dessert")).fetchall()
with engine.begin() as connection:
    item_boisson_pk = connection.execute(db.select([item_table.c.nom_item]).where(item_table.c.type=="boisson")).fetchall()

with engine.begin() as connection:
    for _ in range(15):
      insert_stmt = menu_table.insert().values(
          nom_menu = faker.unique.company(),
          plat = random.choice(item_plat_pk)[0],
          dessert = random.choice(item_dessert_pk)[0],
          boisson = random.choice(item_boisson_pk)[0],
          prix = round(random.uniform(8,15),2)

      )
      connection.execute(insert_stmt)